# Model Prediction

### Stacked Model

In [8]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.linear_model import ElasticNet
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.model_selection import train_test_split, cross_val_score
import warnings
import os
import timeit
import time
import datetime

%matplotlib inline
warnings.filterwarnings('ignore')
SEED=42
CV_FOLDS=10

kc_data = pd.read_csv('./data/ci_i5_ap.csv')

In [9]:
kc_data = kc_data.drop(['id', 'date'], axis=1)
kc_data.head()
kc_train, kc_test, train_Y, test_Y = train_test_split(
    kc_data, kc_data['price'], 
    test_size=0, 
    random_state=SEED
)
kc_train = kc_train.drop('price', axis=1)
kc_test = kc_test.drop('price', axis=1)
scaler = StandardScaler()
kc_train = scaler.fit_transform(kc_train)

In [10]:
from mlxtend.regressor import StackingRegressor
import xgboost as xgb
from sklearn.tree import DecisionTreeRegressor
from sklearn import svm

In [11]:
gbm = xgb.XGBRegressor(
    min_child_weight=9, 
    max_depth=1, 
    objective='reg:linear',
    learning_rate=0.01, 
    seed=SEED, 
    n_estimators=2750, 
    verbose=True
)

svr = svm.SVR(degree=1, C=0.03)

dtr = DecisionTreeRegressor(
    max_depth=12, 
    min_samples_leaf=11,
    min_samples_split=2,
    random_state=SEED
)

### Stacking Combinations

#### Decision Tree + SVM + GBM -> ElasticNet

In [14]:
meta_reg_elastic = ElasticNet(
    random_state=SEED
)
stacked_reg = StackingRegressor(regressors=[dtr, svr, gbm], meta_regressor = meta_reg_elastic)
cv = cross_val_score(stacked_reg, kc_train, train_Y, cv=10, scoring='neg_median_absolute_error')
print(cv.mean())

-46829.583321265876


#### Decision Tree + GBM -> ElasticNet

In [15]:
meta_reg_elastic = ElasticNet(
    random_state=SEED
)
stacked_reg = StackingRegressor(regressors=[dtr, gbm], meta_regressor = meta_reg_elastic)
cv = cross_val_score(stacked_reg, kc_train, train_Y, cv=10, scoring='neg_median_absolute_error')
print(cv.mean())

-47129.0776816442


#### SVM + GBM -> ElasticNet

In [17]:
meta_reg_elastic = ElasticNet(
    random_state=SEED
)
stacked_reg = StackingRegressor(regressors=[svr, gbm], meta_regressor = meta_reg_elastic)
cv = cross_val_score(stacked_reg, kc_train, train_Y, cv=10, scoring='neg_median_absolute_error')
print(cv.mean())

-59525.68511438966


#### Decision Tree + SVM -> ElasticNet

In [18]:
meta_reg_elastic = ElasticNet(
    random_state=SEED
)
stacked_reg = StackingRegressor(regressors=[dtr, svr], meta_regressor = meta_reg_elastic)
cv = cross_val_score(stacked_reg, kc_train, train_Y, cv=10, scoring='neg_median_absolute_error')
print(cv.mean())

-48651.87129091397
